# 比赛介绍

比赛地址： [https://aistudio.baidu.com/aistudio/competition/detail/59](https://aistudio.baidu.com/aistudio/competition/detail/59)

飞桨(PaddlePaddle)以百度多年的深度学习技术研究和业务应用为基础，是中国首个开源开放、技术领先、功能完备的产业级深度学习平台。更多飞桨资讯，点击此处查看。

飞桨常规赛由百度飞桨于2019年发起，面向全球AI开发者，赛题范围广，涵盖领域多。常规赛旨在通过长期发布的经典比赛项目，为开发者提供学习锻炼机会。

# 赛题介绍

本次赛题数据集由学术网络图构成，该图会给出每个节点的特征，以及节点与节点间关系（训练集节点的标注结果已给出）。

数据：

输入数据是一整张图，该图包含1647958条有向边，130644个节点。

提供的数据文件说明：

| 数据集 | 简介 |
| -------- | -------- | 
| edges.csv    | 边数据：用于标记论文间引用关系    | 
|feat.npy|节点数据：每个节点含100维特征|
|train.csv|训练集数据|
|test.csv|测试集数据|

- edges.csv，用于标记论文引用关系，为无向图，且由两列组成，没有表头。

- feat.npy， Numpy格式存储的节点特征矩阵，Shape为(130644, 100)，可以用numpy.load(“feat.npy”)

- train.csv，包含两个字段，nid 和 label。

- test.csv，只包含 nid


# 提交结果介绍

提交内容与格式
最终提交的submission.csv 格式如下：

|字段	| 说明 |
| --- | --- |
|nid  | 测试集节点在图上的id | 
|label | 测试集的节点类别|

提交样例

|nid | label |
| -- | --- |
|2  | 34 |
|3	| 1 |
|4 | 5 |
|… | … |

# 项目杂谈

环境： AI studio; **PaddlePaddle 2.1.0**;  **PGL 2.1.5**

在七月份的这次比赛中，我们仅使用了单模型，没有进行模型融合，取得的分数为0.74777，获得第七名。 比较遗憾的是，模型文件被覆盖掉了。因此无法复现出原来的结果。但是我们在八月初跑出了0.74938的分数，我们将提供跑出这个结果的预训练模型

项目贡献：我们的项目是第一个使用paddle2.1.0去打这个比赛的，因此我们的整个项目都是基于动态图模型，相较于paddle1.8更容易理解和修改。

不足之处：由于时间关系，我们仅利用了PGL提供的一些基本图卷积算法，例如GCN，GAT，APPNP，GCNII等，没有使用更复杂的模型，比如榜首使用UniMP算法，因为没有找到现成的动态图实现，加上时间不够，也没有机会去复现这个算法，不然结果应该会更好。但是我们应该会在八月份实现一个UniMP算法的动态图版本，可以期待一下。




# 项目介绍

我们将整个项目目录结构如下:

```
.
├── config.py       // 模型的基本配置参数
├── model          // 保存训练好的模型
├── network         // 使用的图神经网络算法
│   ├── appnp.py
│   ├── gat.py
│   ├── gcnii.py
│   ├── gcn.py
│   ├── __init__.py
│   └── transformer.py
├── result           // 每个模型跑出的结果文件
│   ├── APPNP.csv
│   ├── GAT.csv
│   ├── gat-lstm.csv
│   ├── gcn.csv
│   ├── GCN.csv
│   ├── gcn-lstm.csv
│   ├── ResGAT.csv
│   └── sage.csv
├── submission.csv   // 提交的结果文件
├── train.py         // 训练文件
└── util                  // 数据集加载以及准确率计算和模型创建函数
    ├── __init__.py
    ├── load_dataset.py
    └── tools.py
```

整个项目已经完全结构化，如果需要定义自己的模型，可以将模型加入到network文件夹下，然后在 ./util/tools.py 中定义模型的初始化。 最后在train.py 中调用config, model = get_config_model('model_name') 即可。

# 代码

In [ ]:
!pip install pgl

结果复现：直接运行下面的cell即可，模型会在当前目录下生成submission.csv文件

如果要重新训练，需要将main函数中的第一个False该为Ture。

In [3]:
import paddle
import paddle.nn.functional as F 

from util import *
from network import *
from config import Config

import pandas as pd
import os


# 模型训练
def train(node_index, node_label, model, graph, criterion, optim):
    model.train()
    pred = model(graph, graph.node_feat["feat"])
    pred = paddle.gather(pred, node_index)
    loss = criterion(pred, node_label)
    loss.backward()

    pred = paddle.argmax(F.softmax(pred, axis=1), axis=1).numpy()
    node_label = node_label.flatten().numpy()
    acc = calc_accuracy(pred, node_label)
    optim.step()
    optim.clear_grad()

    return loss.numpy()[0], acc


# 模型评估
def evaluate(node_index, node_label, model, graph, criterion):
    model.eval()
    pred = model(graph, graph.node_feat["feat"])
    pred = paddle.gather(pred, node_index)
    loss = criterion(pred, node_label)
    
    pred = paddle.argmax(F.softmax(pred, axis=1), axis=1).numpy()
    node_label = node_label.flatten().numpy()
    acc = calc_accuracy(pred, node_label)

    return loss.numpy()[0], acc


# 模型预测
def predict(model, config, graph):
    # 加载预训练模型参数
    model.set_dict(paddle.load(os.path.join(config.model_path, config.model_name + '.params')))
    model.eval()
    
    # 加载数据集并进行预测
    test_ids = load_test(config.test)
    pred = model(graph, graph.node_feat["feat"])
    pred = paddle.gather(pred, test_ids)
    pred = paddle.argmax(F.softmax(pred, axis=1), axis=1).numpy()

    # 保存预测结果文件
    df = pd.DataFrame({'nid': test_ids, 'label': pred})
    df.to_csv(os.path.join(config.result_path, config.model_name + '.csv'), index=False)
    df.to_csv('submission.csv', index=False)


def main(is_train=True, is_predict=True):
    
    # 获取配置信息和模型
    config, model = get_config_model('ResGAT')

    print('train model: ', config.model_name)
    

    # 定义损失函数，优化器
    criterion = paddle.nn.CrossEntropyLoss()
    optimizer = paddle.optimizer.AdamW(parameters=model.parameters(), learning_rate=config.lr)

    # 获取训练、测试数据
    graph = build_graph(config)
    graph.tensor()
    train_ids, train_labels, eval_ids, eval_labels = load_train(config.train)

    best_acc = 0.0
    if is_train:
        print('start training...')
        for epoch in range(config.epoch):
            train_loss, train_acc = train(train_ids, train_labels, model, graph, criterion, optimizer)
            eval_loss, eval_acc = evaluate(eval_ids, eval_labels, model, graph, criterion)

            if epoch % 20 == 0:
                print('epoch: {} train_loss: {:.4f} train_acc: {:.4f} eval_loss: {:.4f} eval_acc: {:.4f}'.format(
                        epoch, train_loss, train_acc, eval_loss, eval_acc))

            # 每找到一个最好的模型就保存下来
            if eval_acc > best_acc:
                best_acc = eval_acc
                paddle.save(model.state_dict(), os.path.join(config.model_path, config.model_name + '.params'))
    
    if is_predict:
        print('start predicting...')
        predict(model, config, graph)


if __name__ == '__main__':
    # 训练完成之后立即对结果进行预测
    main(False, True)


train model:  ResGAT


FileNotFoundError: [Errno 2] No such file or directory: '/home/aistudio/data/data101014/feat.npy'